In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from sklearn.model_selection import train_test_split
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader,random_split
from torch.utils.data.sampler import SubsetRandomSampler
import time
import os
from PIL import Image
import glob
import cv2
from sklearn.preprocessing import LabelBinarizer
import pickle


use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Using CUDA


In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive"

ls: cannot access '/content/gdrive/My Drive': No such file or directory


In [0]:
!unzip '/content/gdrive/My Drive/test.zip' -d '/content/gdrive/My Drive/daataa'

In [0]:

#VGG takes imgs in 224 *224 size
train_transform = transforms.Compose([transforms.RandomRotation(10),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.Resize(224),
                                      transforms.ToTensor(),                                      
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], \
                                                           std=[.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], \
                                                           std=[.229, 0.224, 0.225])])


In [0]:

os.chdir('/content/gdrive/My Drive/daataa')

In [0]:
from copy import copy
shuffle_dataset = True
random_seed = 55
validation_split = 0.1
test_split = 0.2

IMG_PATH = os.getcwd() + '/test'
total_dataset = datasets.ImageFolder(IMG_PATH,transform=train_transform)
# try_load = DataLoader(try_dataset,shuffle=True,num_workers=0,batch_size=1)
        
len_tot = total_dataset.__len__()
test_len = int(len_tot * test_split)

train_data , test_data = random_split(total_dataset,[len_tot - test_len,test_len])
train_data , valid_data = random_split(train_data,[train_data.__len__() - test_len , test_len ])

train_data.dataset.transform = train_transform
valid_data.dataset.transform = test_transform
test_data.dataset.transform = test_transform

train_data.__len__()


21660

In [0]:
#creating datsets and dataloaders

batch_size = 40

# train_sampler = SubsetRandomSampler(train_indices)
# test_sampler = SubsetRandomSampler(test_indices)
# valid_sampler = SubsetRandomSampler(valid_indices)

train_load = DataLoader(train_data, batch_size=batch_size,num_workers=12)
valid_load = DataLoader(valid_data, batch_size=batch_size,num_workers=10)
test_load = DataLoader(test_data, batch_size=batch_size,num_workers=10)

# a,b =  next(iter(train_load))




In [0]:
#This is used to one hot encode the labels during training - NOT NEEDED
# idx = b
# idx.unsqueeze(1)
def encode(arr):
    
    return torch.zeros(len(arr), arr.max()+1).scatter_(1, arr.unsqueeze(1), 1.)

    
class_names = total_dataset.classes
class_names

['ANGRY', 'DISGUST', 'FEAR', 'HAPPY', 'NEUTRAL', 'SAD', 'SURPRISE']

In [0]:
#load VGG16_bn model - this one finally worked
model = models.vgg16_bn(pretrained=True)

#freezing the weights
for param in model.parameters():
    param.requires_grad = False

#modifying the last layer
num_in = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # take all but last layer
features.extend([nn.Linear(num_in,len(class_names))])#custom last layer
model.classifier = nn.Sequential(*features)# attaching it back to classifier

model.to(device)



Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:06<00:00, 91490507.27it/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [0]:
#training only the classfier

for param in model.classifier.parameters():
    param.requires_grad = True

# for param in model.parameters():
#     print(param.name, param.requires_grad)


In [0]:
#Time to train the model

#defining  variables
epochs = 50
no_epoch_stop = 5
optimizer = optim.SGD(model.parameters(),lr = 0.0001,momentum = 0.8)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=6,gamma=0.1)
criteria = nn.CrossEntropyLoss()
min_valid_loss = np.Inf
model.to(device)
start = time.time()

try :
  
  model.load_state_dict(torch.load('checkpoint2.pt'))
  
except:
  pass

try:
  min_valid_loss = pickle.load(open('min_val2.pkl','rb'))
except:
  pass
#=====================================================================
epoch_dec = 0
for epoch in range(epochs):
    print('epoch {}/{}:'.format(epoch,epochs))
    
    train_err = 0.0
    valid_err = 0.0
    scheduler.step()
    
    #training part
    model.train()
    i=0# to keep track
    for data , label in train_load:
        
        if (i)%500 ==0:
            print('{}% points done'.format(i/len(train_load)*100))
#       label = encode(label) # CEL takes batch size array with class index..no need to encode
        label = label.long()
        data, label = data.to(device) , label.to(device)
        
        optimizer.zero_grad()
        
        log_ps = model(data)

        loss = criteria(log_ps,label)
        
        loss.backward()
        optimizer.step()
        
        train_err += loss.item()
        i +=1
    
    #validation part
    model.eval()
    for data , label in valid_load:
        
#         label = encode(label)
        label = label.long()
        data, label = data.to(device) , label.to(device)
        
        out = model(data)
        loss = criteria(out, label)
        
        valid_err += loss.item()
        
    #avg the losses
    
    train_err /= len(train_load)
    valid_err /= len(valid_load)
    
    if valid_err < min_valid_loss:
        print('saving')
        min_valid_loss = valid_err
        pickle.dump(min_valid_loss,open('min_val2.pkl','wb'))
        torch.save(model.state_dict(),'checkpoint2.pt')
        epoch_dec = 0
        
        
        
    
    else:
        
        epoch_dec +=1
        print('epoch dec',epoch_dec)
        if epoch_dec == no_epoch_stop:
            print('early stopping')
            model.load_state_dict(torch.load('checkpoint2.pt'))
            break
    
    print('='*50)
    print('training loss:' + str(train_err))
    print('validation loss:' + str(valid_err))
    print('='*50)
        
end_t = time.time()

print('total time taken:{}'.format(end_t - start))
        
        
        
        

    
    


epoch 0/50:
0.0% points done
92.25092250922509% points done
saving
training loss:1.0732236695905453
validation loss:0.9536862386524348
epoch 1/50:
0.0% points done
92.25092250922509% points done
epoch dec 1
training loss:1.0706952701415522
validation loss:0.9540228033592688
epoch 2/50:
0.0% points done
92.25092250922509% points done
epoch dec 2
training loss:1.0694460222202034
validation loss:0.9540183656123462
epoch 3/50:
0.0% points done
92.25092250922509% points done
epoch dec 3
training loss:1.0726585052770001
validation loss:0.9544167775475518
epoch 4/50:
0.0% points done
92.25092250922509% points done
epoch dec 4
training loss:1.0694926543869216
validation loss:0.9547275057813739
epoch 5/50:
0.0% points done
92.25092250922509% points done
epoch dec 5
early stopping
total time taken:2906.6634101867676


In [0]:
#loading the pre-trained model

model.load_state_dict(torch.load('checkpoint2.pt'))
criteria = nn.CrossEntropyLoss()
accuracy = 0.0
test_loss = 0.0
correct =0


#checking for accuracy
for inputs, classes in test_load:
  inputs = inputs.to(device)
  output = model.forward(inputs)
  classes = classes.to(device)
  #         test_loss += criteria(output, classes).item()

  ps = torch.exp(output)

  pred = output.data.max(1, keepdim=True)[1]
  correct += pred.eq(classes.data.view_as(pred)).cpu().sum()
#   print(correct)





print(' Accuracy: {}/{} ({:.3f}%)\n'.format(
        correct,len(test_load.dataset), 100. * correct / len(test_load.dataset)))


KeyboardInterrupt: ignored

In [0]:
#training it on animated faces dataset for generalization
# !unzip -u '/content/gdrive/My Drive/test2.zip' -d '/content/gdrive/My Drive/daataa2'
# !unzip -q '/content/gdrive/My Drive/daataa2/test2/HAPPY.zip' -d  '/content/gdrive/My Drive/daataa2/test2/HAPPY'
!ls '/content/gdrive/My Drive/daataa2/test2'


ANGRY  DISGUST	FEAR  HAPPY  NEUTRAL  SAD  SURPRISE


In [0]:
os.chdir('/content/gdrive/My Drive/daataa2')

In [0]:
#VGG takes imgs in 224 *224 size
train_transform = transforms.Compose([transforms.RandomRotation(10),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.Resize(224),
                                      transforms.ToTensor(),                                      
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], \
                                                           std=[.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], \
                                                           std=[.229, 0.224, 0.225])])


In [0]:
from copy import copy
shuffle_dataset = True
random_seed = 55
validation_split = 0.1
test_split = 0.2

IMG_PATH = os.getcwd() + '/test2'
total_dataset = datasets.ImageFolder(IMG_PATH,transform=train_transform)
# try_load = DataLoader(try_dataset,shuffle=True,num_workers=0,batch_size=1)
        
len_tot = total_dataset.__len__()
test_len = int(len_tot * test_split)

train_data , test_data = random_split(total_dataset,[len_tot - test_len,test_len])
train_data , valid_data = random_split(train_data,[train_data.__len__() - test_len , test_len ])

train_data.dataset.transform = train_transform
# valid_data.dataset.transform = test_transform
# test_data.dataset.transform = test_transform

train_data.__len__()

33460

In [0]:
#creating datsets and dataloaders

batch_size = 40

# train_sampler = SubsetRandomSampler(train_indices)
# test_sampler = SubsetRandomSampler(test_indices)
# valid_sampler = SubsetRandomSampler(valid_indices)

train_load = DataLoader(train_data, batch_size=batch_size,num_workers=12)
# valid_load = DataLoader(valid_data, batch_size=batch_size,num_workers=10)
# test_load = DataLoader(test_data, batch_size=batch_size,num_workers=10)

a,b =  next(iter(train_load))

In [0]:
#This is used to one hot encode the labels during training - NOT NEEDED
# idx = b
# idx.unsqueeze(1)
def encode(arr):
    
    return torch.zeros(len(arr), arr.max()+1).scatter_(1, arr.unsqueeze(1), 1.)

    
class_names = total_dataset.classes
class_names

['ANGRY', 'DISGUST', 'FEAR', 'HAPPY', 'NEUTRAL', 'SAD', 'SURPRISE']

In [0]:
#load VGG16_bn model - this one finally worked
model = models.vgg16_bn(pretrained=True)

#freezing the weights
for param in model.parameters():
    param.requires_grad = False

#modifying the last layer
num_in = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # take all but last layer
features.extend([nn.Linear(num_in,len(class_names))])#custom last layer
model.classifier = nn.Sequential(*features)# attaching it back to classifier

#training only the classfier

for param in model.classifier.parameters():
    param.requires_grad = True

# for param in model.parameters():
#     print(param.name, param.requires_grad)

model.to(device)


Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:05<00:00, 100581150.98it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [0]:

#defining  variables
epochs = 20
no_epoch_stop = 5
optimizer = optim.SGD(model.parameters(),lr = 0.0001,momentum = 0.8)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=6,gamma=0.1)
criteria = nn.CrossEntropyLoss()
min_valid_loss = np.Inf
model.to(device)
start = time.time()

# try :
  
#   model.load_state_dict(torch.load('checkpoint.pt'))
  
# except:
#   pass

# try:
#   min_valid_loss = pickle.load(open('min_val.pkl','rb'))
# except:
#   pass
#=====================================================================
epoch_dec = 0
for epoch in range(epochs):
    print('epoch {}/{}:'.format(epoch,epochs))
    
    train_err = 0.0
    valid_err = 0.0
    scheduler.step()
    
    #training part
    model.train()
    i=0# to keep track
    for data , label in train_load:
        
        if (i)%500 ==0:
            print('{}% points done'.format(i/len(train_load)*100))
#       label = encode(label) # CEL takes batch size array with class index..no need to encode
        label = label.long()
        data, label = data.to(device) , label.to(device)
        
        optimizer.zero_grad()
        
        log_ps = model(data)

        loss = criteria(log_ps,label)
        
        loss.backward()
        optimizer.step()
        
        train_err += loss.item()
        i +=1
    
    #validation part
    model.eval()
    for data , label in valid_load:
        
#         label = encode(label)
        label = label.long()
        data, label = data.to(device) , label.to(device)
        
        out = model(data)
        loss = criteria(out, label)
        
        valid_err += loss.item()
        
    #avg the losses
    
    train_err /= len(train_load)
    valid_err /= len(valid_load)
    
    if valid_err < min_valid_loss:
        print('saving')
        min_valid_loss = valid_err
        pickle.dump(min_valid_loss,open('min_val.pkl','wb'))
        torch.save(model.state_dict(),'checkpoint.pt')
        epoch_dec = 0
        
        
        
    
    else:
        
        epoch_dec +=1
        print('epoch dec',epoch_dec)
        if epoch_dec == no_epoch_stop:
            print('early stopping')
            model.load_state_dict(torch.load('checkpoint.pt'))
            break
    
    print('='*50)
    print('training loss:' + str(train_err))
    print('validation loss:' + str(valid_err))
    print('='*50)
        
end_t = time.time()

print('total time taken:{}'.format(end_t - start))
        
        

epoch 0/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.2805116270160305
validation loss:0.09720499137930545
epoch 1/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.12129087339343421
validation loss:0.04653388012774742
epoch 2/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.07584682585428977
validation loss:0.02810248809676337
epoch 3/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.053757787220349164
validation loss:0.01919593255112355
epoch 4/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.041834415613118396
validation loss:0.014032140447621277
epoch 5/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.03668964199672855
validation loss:0.013636892464505942
epoch 6/20:
0.0% points done
59.737156511350065% points done
saving
training loss:0.03553201059789676
validation loss:0.013234882784079373
epoch 7/20:
0.0% points done
5

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self

KeyboardInterrupt: ignored

In [0]:
#loading the pre-trained model

model.load_state_dict(torch.load('checkpoint.pt'))
criteria = nn.CrossEntropyLoss()
accuracy = 0.0
test_loss = 0.0
correct =0


#checking for accuracy
for inputs, classes in test_load:
  inputs = inputs.to(device)
  output = model.forward(inputs)
  classes = classes.to(device)
  #         test_loss += criteria(output, classes).item()

  ps = torch.exp(output)

  pred = output.data.max(1, keepdim=True)[1]
  correct += pred.eq(classes.data.view_as(pred)).cpu().sum()
#   print(correct)





print(' Accuracy: {}/{} ({:.3f}%)\n'.format(
        correct,len(test_load.dataset), 100. * correct / len(test_load.dataset)))

RuntimeError: ignored